# Batch-send emails using a csv table

In [11]:
import smtplib, email, ssl
import pandas as pd
import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from bs4 import BeautifulSoup as bs
from email.utils import formatdate
from os.path import basename # extract basename for the file path
from dotenv import load_dotenv
import os

In [10]:
load_dotenv()

EMAIL_HOST_PASSWORD =os.environ.get('EMAIL_HOST_PASSWORD')
EMAIL_HOST = os.environ.get('EMAIL_HOST')
EMAIL_HOST_USER = os.environ.get('EMAIL_HOST_USER')
EMAIL_PORT = os.environ.get('EMAIL_PORT')

smtp.zoho.com


In [26]:
# email contents
subject = '志愿者证书-华人抗体协会感谢你的支持!'
date = datetime.date.strftime(datetime.date.today(), '%Y-%m-%d')


# Load dataset
df = pd.read_csv('example/test.csv')[:1] # load 40 rows each time is OK. Sometimes server disconnects but you can pick up from where you're.

df


,Unique ID,First Name,Last Name,Name,Start,Status,End,Email
0,Volun0104,Zhuxuan,Xu,Zhuxuan Xu,2018.0,Active,2020.0,kellen.xu@rutgers.edu


In [27]:
for id, name, email in zip(df['Unique ID'], df['Name'], df['Email']):

    # get file name and path
    fn = id + ' - ' + name +'.pdf'
    fp = 'example/test/' + fn

    # message
    message = '尊敬的'+name+ """:<br><br>
希望你一切安好。2020年就要过去了。虽然有疫情的挑战，协会今年依然有长足的进展。而这一切与你的大力支持密不可分。为了感谢你对协会的付出，协会制作了志愿者证书以聊表寸心。<u>证书请见附件</u>。 祝你在2021年万事如意! <br><br>
华人抗体协会<br>""" + date

    # Parse the html content into soup object
    with open('template/2021-new-year.html') as temp:
        soup=bs(temp)

        # add message to soup
        soup.find("p", id="message").append(message)
        soup=soup.prettify(formatter=None)
        soup = soup.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace("\\'", " \' ").replace('\'re', " are" )

    # message
    msg = MIMEMultipart() # must reinitialize for each email
    msg.set_charset('utf8')
    msg['From'] = EMAIL_HOST_USER
    msg['Subject'] = subject
    msg['Date'] = formatdate(localtime=True)
    msg['To'] = email
    msg.attach(MIMEText(str(soup), 'html'))

    # attach
    with open(fp, "rb") as fil:
        part = MIMEApplication(
            fil.read(),
            Name=basename(fp)
        )
    part['Content-Disposition'] = 'attachment; filename="%s"' % basename(fp)
    msg.attach(part)

    # send
    SERVER = smtplib.SMTP(EMAIL_HOST, EMAIL_PORT)
    SERVER.connect(EMAIL_HOST, EMAIL_PORT)
    SERVER.starttls()
    SERVER.login(EMAIL_HOST_USER, EMAIL_HOST_PASSWORD)
    SERVER.sendmail(
            EMAIL_HOST_USER,
            email,
            msg.as_string()
        )
    print('Email with attachment '+ fn + ' sent to ' + email) # print this, in case server disconnects and you can resume from where it is left.

Email with attachment Volun0104 - Zhuxuan Xu.pdf sent to kellen.xu@rutgers.edu
